<a href="https://colab.research.google.com/github/fathi99/Fathi_portfolio/blob/main/K_Means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LOAD DATA FROM GOOGLE DRIVE TO PHYTON

In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random



In [ ]:
#Loading data Option 3: from google drive 

#Load data from google drive 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#user authentication
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#accessing the train file (make sure you have the file and copy the file id correctly)
downloaded = drive.CreateFile({'id':"1pNdtFhrHdyH1dxAiayOfSoB2rqf7VlaV"})
downloaded.GetContentFile('movies.csv') 

downloaded = drive.CreateFile({'id':"14aiS_7HCSghxCE2ynN9WGn7_xrzlqeRl"})
downloaded.GetContentFile('ratings.csv') 


In [ ]:
movies=pd.read_csv('https://raw.githubusercontent.com/hafizhantesting/datamining/main/movies.csv')
movies.fillna(0,inplace=True)
	
movies.title = movies.title.astype(str)
movies.genres = movies.genres.astype(str)
movies.dtypes


movieId          int64
title           object
year           float64
genres          object
Unnamed: 4     float64
Unnamed: 5     float64
Unnamed: 6     float64
Unnamed: 7     float64
Unnamed: 8     float64
Unnamed: 9     float64
Unnamed: 10    float64
Unnamed: 11    float64
title(year)     object
dtype: object

In [ ]:
ratings=pd.read_csv('ratings.csv')
ratings=ratings.drop('timestamp',axis = 1)
ratings.rating = ratings.rating.astype(int)
ratings.fillna(0,inplace=True)
ratings.dtypes

userId     int64
movieId    int64
rating     int64
dtype: object

In [ ]:
data = pd.merge(ratings , movies , how='outer', on='movieId')
data.head()

,userId,movieId,rating,title,genres
0,1.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15.0,1,2.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [ ]:
# Data Processing
# Converting Genres into different columns 
# Here we just create columns and put there initial value as 0
x = movies.genres
a = list()
for i in x:
    abc = i
    a.append(abc.split('|'))
a = pd.DataFrame(a)   
b = a[0].unique()
for i in b:
    data[i] = 0
for i in b:
    data.loc[data['genres'].str.contains(i), i] = 1
    data.head(2)

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [ ]:
data = data.drop(['genres','title'],axis =1)
data.dropna(inplace= True )
data.head()

,userId,movieId,rating,Adventure,Comedy,Action,Drama,Crime,Children,Mystery,Animation,Documentary,Thriller,Horror,Fantasy,Western,Film-Noir,Romance,Sci-Fi,Musical,War,0,(no genres listed)
0,1.0,1,4.0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0
1,5.0,1,4.0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0
2,7.0,1,4.0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0
3,15.0,1,2.0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0
4,17.0,1,4.0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
#By different meathods I found 8 cluster are better 
kmeanModel = KMeans(n_clusters=8)
kmeanModel.fit(data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
data['Cluster'] = kmeanModel.labels_
data['Cluster'].sample(n=10)

86280    0
42957    5
94776    0
68089    5
62339    0
4863     0
31346    0
80237    0
64435    0
18310    0
Name: Cluster, dtype: int32

In [ ]:
e = []
def fi(group):
    a = pd.DataFrame(group)
    b = pd.DataFrame(a['Cluster'].value_counts())
    d = a.index 
    c = [a['movieId'][d[0]],int(b.idxmax())]
    e.append(c)
data.groupby("movieId").apply(lambda x: fi(x))
e = pd.DataFrame(e)
e.rename(columns = {0:'movieId',1:'Cluster'},inplace=True)
e.drop_duplicates(inplace=True)
e.head(10)

,movieId,Cluster
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
5,6,0
6,7,0
7,8,0
8,9,0
9,10,0


In [ ]:
data1 = pd.read_csv('movies.csv')
data1.title = data1['title'].str.replace(r"\(.*\)","")
new_data = pd.merge(e , data1 , how='outer', on='movieId')
new_data.fillna(random.randint(0,8),inplace=True)
new_data.head()

,movieId,Cluster,title,genres
0,1,0.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,0.0,Jumanji,Adventure|Children|Fantasy
2,3,0.0,Grumpier Old Men,Comedy|Romance
3,4,0.0,Waiting to Exhale,Comedy|Drama|Romance
4,5,0.0,Father of the Bride Part II,Comedy


In [ ]:
def select_c():
    global l
    print('Select The Movies Id you would like to watch:')
    l=[]
    for i in range(15):
        l.append(random.randint(0,3883))
    for i in l:
        print(new_data['movieId'][i] , new_data['title'][i],sep='--->')
    print('--------------------------------------------------------------------')
    l = int(input())
    l = new_data['Cluster'][new_data.movieId == l]

In [ ]:
# This is the main function which recommend you movies
def main():
    ans = False
    while not ans:
        select_c()
        print(new_data['title'][new_data.Cluster == int(l)].sample(n=10))
        print('--------------------------------------------------------------------')
        ans =True

In [ ]:
main()

Select The Movies Id you would like to watch:
4857--->Fiddler on the Roof 
4958--->Behind Enemy Lines 
1581--->Out to Sea 
1083--->Great Race, The 
3858--->Cecil B. DeMented 
2771--->Brokedown Palace 
4521--->Little Nikita 
2028--->Saving Private Ryan 
2232--->Cube 
1754--->Fallen 
869--->Kansas City 
569--->Little Big League 
1964--->Klute 
4521--->Little Nikita 
1020--->Cool Runnings 
--------------------------------------------------------------------
3056                   Ernest Goes to Camp 
3164                                Driven 
4321                              Identity 
5084                          Mean Streets 
4824                  To Have and Have Not 
1881                           October Sky 
4443                                  Camp 
861                       Private Benjamin 
327     Mrs. Parker and the Vicious Circle 
4735                 Juliet of the Spirits 
Name: title, dtype: object
--------------------------------------------------------------------
Do yo

KeyboardInterrupt: ignored